# Web Mining and Applied NLP (44-620)

## Requests, JSON, and NLP

### Student Name: Jason Ballard

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

Make sure you have [installed spaCy and its pipeline](https://spacy.io/usage#quickstart) and [spaCyTextBlob](https://spacy.io/universe/project/spacy-textblob)

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

This assignment requires that you write additional files (either JSON or pickle files); make sure to submit those files in your repository as well.

1. The following code accesses the [lyrics.ovh](https://lyricsovh.docs.apiary.io/#reference/0/lyrics-of-a-song/search) public api, searches for the lyrics of a song, and stores it in a dictionary object.  Write the resulting json to a file (either a JSON file or a pickle file; you choose). You will read in the contents of this file for future questions so we do not need to frequently access the API.

## Dependencies 

In [18]:
import requests
import json
import pickle
from textblob import TextBlob
from spacytextblob import spacytextblob
import spacy
#result = json.loads(requests.get('https://api.lyrics.ovh/v1/They Might Be Giants/Birdhouse in your soul').text)

2. Read in the contents of your file.  Print the lyrics of the song (not the entire dictionary!) and use spaCyTextBlob to perform sentiment analysis on the lyrics.  Print the polarity score of the sentiment analysis. Given that the range of the polarity score is `[-1.0,1.0]` which corresponds to how positive or negative the text in question is, do you think the lyrics have a more positive or negative connotaion?  Answer this question in a comment in your code cell.

## Question 1

In [ ]:
# Load spaCy model and add TextBlob component
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

def save_lyrics_to_file(artist, song, filename):
    """
    Fetches lyrics from API and saves them to a file.
    Returns True if successful, False otherwise.
    """
    try:
        url = f'https://api.lyrics.ovh/v1/They Might Be Giants/Birdhouse in your soul'
        response = requests.get(url)
        result = json.loads(response.text)
        
        # Check if lyrics exist in the response
        if 'lyrics' not in result:
            print("Lyrics not found.")
            return False
        
        # Save to file (using pickle in this case)
        with open(filename, 'wb') as f:
            pickle.dump(result, f)
        return True
    except Exception as e:
        print(f"Error: {e}")
        return False

def analyze_lyrics(filename):
    """
    Reads lyrics from file and performs sentiment analysis.
    Returns polarity score.
    """
    try:
        # Read the lyrics from file
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        
        # Ensure 'lyrics' key exists
        if 'lyrics' not in data:
            print("No lyrics found in the file.")
            return None

        # Process with spaCy
        doc = nlp(data['lyrics'])
        polarity = doc._.blob.polarity
        
        # Print polarity score
        print(f"Sentiment Polarity Score: {polarity}")
        
        # Interpret the score
        if polarity > 0:
            print("The lyrics have a positive connotation.")
        elif polarity < 0:
            print("The lyrics have a negative connotation.")
        else:
            print("The lyrics have a neutral connotation.")
            
        return polarity
        
    except Exception as e:
        print(f"Error: {e}")
        return None

# Example usage:
filename = 'song_lyrics.pkl'

# First, save the lyrics
if save_lyrics_to_file('They Might Be Giants', 'Birdhouse in Your Soul', filename):
    # Then analyze them
    polarity = analyze_lyrics(filename)

#Comment 
# Sentiment Polarity Score: 0.04505208333333333
# The lyrics have a positive connotation.


Sentiment Polarity Score: 0.04505208333333333
The lyrics have a positive connotation.


## Question 2 

Write a function that takes an artist, song, and filename, accesses the lyrics.ovh api to get the song lyrics, and writes the results to the specified filename.  Test this function by getting the lyrics to any four songs of your choice and storing them in different files.

In [21]:
def fetch_and_save_lyrics(artist, song, filename):
    """
    Fetches lyrics from the lyrics.ovh API and writes the results to the specified filename.
    
    Args:
        artist (str): The name of the artist.
        song (str): The name of the song.
        filename (str): The name of the file to save the lyrics.

    Returns:
        bool: True if lyrics were successfully saved, False otherwise.
    """
    try:
        # Create the API URL
        url = f'https://api.lyrics.ovh/v1/{artist}/{song}'
        
        # Fetch lyrics from the API
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch lyrics for {artist} - {song}. Status Code: {response.status_code}")
            return False
        
        # Parse the JSON response
        result = response.json()
        
        # Check if 'lyrics' key exists
        if 'lyrics' not in result:
            print(f"Lyrics not found for {artist} - {song}.")
            return False
        
        # Save the lyrics to the specified file
        with open(filename, 'wb') as file:
            pickle.dump(result, file)
        print(f"Lyrics saved to {filename}.")
        return True
    except Exception as e:
        print(f"Error fetching or saving lyrics: {e}")
        return False

# Test the function with four songs
songs = [
    ("Taylor Swift", "Love Story", "love_story.pkl"),
    ("Coldplay", "Yellow", "yellow.pkl"),
    ("Adele", "Hello", "hello.pkl"),
    ("Ed Sheeran", "Shape of You", "shape_of_you.pkl")
]

for artist, song, filename in songs:
    fetch_and_save_lyrics(artist, song, filename)


Lyrics saved to love_story.pkl.
Lyrics saved to yellow.pkl.
Lyrics saved to hello.pkl.
Lyrics saved to shape_of_you.pkl.


## Question 3 

Write a function that takes the name of a file that contains song lyrics, loads the file, performs sentiment analysis, and returns the polarity score.  Use this function to print the polarity scores (with the name of the song) of the three files you created in question 3.  Does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?  Answer the questions in either a comment in the code cell or a markdown cell under the code cell.